# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Experiment, Model, Webservice
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep

import pandas as pd

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.
We will be using the 'Credit Card customers' dataset from Kaggle and predicting if a customer is likely to churn.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'demo-automl-experiment'

experiment=Experiment(ws, experiment_name)

# prepare the datastore to upload data
datastore = ws.get_default_datastore()

# set the upload location to target_path in datastore
datastore.upload(src_dir='./', target_path = 'data')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EB9M4V329 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Uploading an estimated of 2 files
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 1 files out of an estimated total of 2
Uploading ./.ipynb_aml_checkpoints/automl-checkpoint2020-11-16-15-7-46.ipynb
Uploaded ./.ipynb_aml_checkpoints/automl-checkpoint2020-11-16-15-7-46.ipynb, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_eb3af9d69fbf4e1ab2e3c246f4812477

In [4]:
# vm_size = "Standard_D2_V2"
# compute_name = "demo-cluster"
# compute_min_nodes = 0
# compute_max_nodes = 4

# if compute_name in ws.compute_targets:
#     compute_target = ws.compute_targets[compute_name]
#     if compute_target and type(compute_target) is AmlCompute:
#         print("")

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
compute_name = "demo-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.

Running


In [5]:
dataset = Dataset.get_by_name(ws, name = "bankchurners")
# dataset = data_prep.get_dataset(ws)
dataset

{
  "source": [
    "('workspaceblobstore', 'UI/12-16-2020_030925_UTC/BankChurners.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "ac326dd1-9965-4cfa-95e8-a2a4b204e220",
    "name": "bankchurners",
    "version": 1,
    "description": "Bank churn dataset",
    "workspace": "Workspace.create(name='quick-starts-ws-130726', subscription_id='de47103e-2da6-4f5e-88fc-d18b27fd249b', resource_group='aml-quickstarts-130726')"
  }
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings and config here

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    task = "classification",
    compute_target = "demo-cluster",
    max_concurrent_iterations = 5,
    training_data = dataset,
    primary_metric = "accuracy",
    label_column_name = "Attrition_Flag",
    n_cross_validations = 3,
    enable_early_stopping = True
)

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on demo-cluster with default configuration
Running on remote compute: demo-cluster
Parent Run ID: AutoML_b672c35b-2c74-4abc-9f92-b03a17d57ddd

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a f

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output = True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|1627                             |Attrited Customer                |10127                                 |
+---------------------------------+----------

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
# Retrieve and save the best AutoML model
best_run, best_model = remote_run.get_output()
best_metrics = best_run.get_metrics()

In [12]:
#TODO: Save the best model
model = best_run.register_model(model_name = "best_automl_model", model_path = "./outputs/")
# model = best_automl_run.register_model(model_name = 'best_automl_model', model_path = 'outputs/model.pkl', model_framework=Model.Framework.SCIKITLEARN)
print(best_run)

Run(Experiment: demo-automl-experiment,
Id: AutoML_b672c35b-2c74-4abc-9f92-b03a17d57ddd_44,
Type: azureml.scriptrun,
Status: Completed)


In [13]:
print(best_metrics["accuracy"])
print(best_model._final_estimator)
# print(best_model)

0.9725487098472879
PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('25',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f1260ee6b70>),
                                                           ('xgboostclassifier',
                                                            XGBoostClassifier(base_score=0.5,
                                                                              booster='gbtree',
                                                                              colsample_bylevel=1,
                                                                              colsample_bynode=1,
                                                                              colsample_bytree=1,
  

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
# from azureml.core.webservice import LocalWebservice

# Copy the created scoring_file_v_1_0_0.py file from the automl run as score.py

script_name='score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_name)

# Create the inference config
env = best_run.get_environment()

inference_config = InferenceConfig(entry_script=script_file_name, environment=env)

# Deploy the model as a webservice

# local_config = LocalWebservice.deploy_configuration(port=9000)
web_service = Model.deploy(ws, 'model-service', [model], inference_config, overwrite=True)
# local_service = Model.deploy(ws, 'model-service', [auto_ml], inference_config, local_config)
web_service.wait_for_deployment(show_output=True)
web_service.update(enable_app_insights=True)

# rest_service = Model.deploy(ws, 'best-model-service', [model], inference_config=inference_config, overwrite=True)
# rest_service.wait_for_deployment(show_output=True)
# rest_service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import json

data_sample = dataset.to_pandas_dataframe().sample(3)
# y_true = data_sample.pop('legitimate')
test_sample = json.dumps({'data':data_sample.to_dict(orient='records')})
print(test_sample)

# data= ({'data':test_df[0:3].to_dict(orient='records')})
# test_sample=json.dumps(data)
# output= service.run(test_sample)
# print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Print web service logs
service_logs = web_service.get_logs()

for line in service_logs.split('\n'):
    print(line)

# Delete the service
service.delete()